In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import header
paths = header.setup_environment()
import platform
if platform.system()=='Darwin':
    INPUTDIR = '/Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3_Project_Documents/NASAprojects/201602_Rocket_Seismology/DATA/2022_DATA/WellData/MERGED'
else:
    INPUTDIR = os.path.join(paths['new_data'], '03_merge_inventories')
import libWellData as LLE
transducersDF = LLE.get_transducers_dataframe(paths)
display(transducersDF)

# Load in the summary of all files - raw 4 hourly data 
dfall2 = pd.read_csv(os.path.join(INPUTDIR, 'all2.csv'))
#display(dfall2['subdir'])

LLE.qc_dataframe(dfall2)

# Split in baro, 20Hz, 100Hz
all_dataframes = LLE.split_by_subdir(dfall2, verbose=True)
#print(all_dataframes)


In [ ]:
startt = dfall2['datetime'].min()
endt = dfall2['datetime'].max()
thisdate = pd.to_datetime(f'{startt.year}-{startt.month:02d}-{startt.day:02d}')
lod = []
csvfile = 'dailyshifts.csv'
if os.path.isfile(csvfile):
    dfshifts = pd.read_csv('dailyshifts.csv', index_col=None)
    lod = dfshifts.to_dict('records')
    startt = pd.to_datetime(dfshifts.iloc[-1]['datetime'])+pd.Timedelta(hours=24)
    thisdate = pd.to_datetime(f'{startt.year}-{startt.month:02d}-{startt.day:02d}')
while thisdate < endt:
    print(thisdate)

    correctedAllSensorsPSI, dcshifts, sensor_depths = LLE.process_day(dfall2, thisdate, INPUTDIR, transducersDF)
    print('dcshifts=',dcshifts)
    print('sensor_depths=',sensor_depths)
    if isinstance(correctedAllSensorsPSI, pd.DataFrame):
        correctedAllSensorsPSI.to_pickle(f'{thisdate.strftime("%Y%m%d")}.pkl')
    if dcshifts or not sensor_depths.empty:
        dcshifts['datetime']=thisdate.strftime("%Y-%m-%d")
        for key, value in sensor_depths.items():
            dcshifts[key] = value
        lod.append(dcshifts)

        # could just do this once at end of loop - but good to see it build each day
        dfshifts = pd.DataFrame(lod)
        dfshifts.to_csv(csvfile, index=None)
        display(dfshifts)

    thisdate = thisdate + pd.Timedelta(hours=24)

